In [ ]:
!pip3 install -U scipy
!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install transformers==4.19.0
!pip3 install -r requirements.txt
!python3 setup.py install
!pip install einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5 torchaudio
!pip install jiwer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481
Receiving objects: 100% (1481/1481), 53.56 MiB | 26.43 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformer

In [ ]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
from transformers import Wav2Vec2Processor
from torch.cuda.amp import GradScaler, autocast
import IPython
from google.colab import files
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voice

In [ ]:
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00
  2% (2555904 of 100715777) |            | Elapsed Time: 0:00:00 ETA:  00:00:00

Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [ ]:
CUSTOM_VOICE_NAME = "custom"
custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder, exist_ok=True)

# Upload custom voice files
for i, file_data in enumerate(files.upload().values()):
    with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
        f.write(file_data)

Saving 2 (1).wav to 2 (1).wav


In [ ]:
# Load audio and normalize
def load_and_normalize_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    normalized_waveform = (waveform - waveform.mean()) / waveform.std()
    return normalized_waveform, sample_rate

voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
voice_samples = [torch.tensor(vs).clone().detach() for vs in voice_samples]
conditioning_latents = [torch.tensor(cl).clone().detach() for cl in conditioning_latents] if conditioning_latents else []

<ipython-input-6-97a6bbcc3f71>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  voice_samples = [torch.tensor(vs).clone().detach() for vs in voice_samples]


In [ ]:
# Load Wav2Vec2 processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Example text for TTS
text = "Here you can find activities to practise your reading skills. Reading will help you to improve your understanding of the language and build your vocabulary."

# Pick a preset mode
preset = "high_quality"

In [ ]:
from torch.utils.data import Dataset
class TTSDataset(Dataset):
    def __init__(self, text_data, voice_samples, conditioning_latents):
        self.text_data = text_data
        self.voice_samples = voice_samples
        self.conditioning_latents = conditioning_latents

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        try:
            text = self.text_data[idx]
            voice = self.voice_samples[idx]
            latent = self.conditioning_latents[idx]

            # Add print statements to debug
            if text is None or voice is None or latent is None:
                print(f"None found at index {idx}")

            return {
                'text': text,
                'voice': voice,
                'latent': latent
            }
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            return None

# Create a list with text data
text_data = [{'text': text}]

In [ ]:
class SimpleTTSModel(nn.Module):
    def __init__(self):
        super(SimpleTTSModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(24000, 512),  # Assuming input size 24000 (adjust as per your input)
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 24000)  # Adjusted output size to match your TTS requirements
        )

    def forward(self, input_tensor):
        x = self.encoder(input_tensor)
        x = self.decoder(x)
        return x

# Initialize model and move to GPU if available
model = SimpleTTSModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Generate speech with the custom voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

In [ ]:
output_audio_path = f'generated-{CUSTOM_VOICE_NAME}.wav'
# Load and normalize generated audio
waveform_gen, sample_rate_gen = load_and_normalize_audio(output_audio_path)

# Load original voice sample to compare
original_audio_path = os.path.join(custom_voice_folder, '0.wav')
waveform_orig, sample_rate_orig = load_and_normalize_audio(original_audio_path)

#Evaluate model dengan mencari RMSE dan SNR output
min_length = min(waveform_gen.size(1), waveform_orig.size(1))
waveform_gen = waveform_gen[:, :min_length]
waveform_orig = waveform_orig[:, :min_length]
rmse = torch.sqrt(torch.mean((waveform_gen - waveform_orig) ** 2))
print(f"RMSE: {rmse.item()}")
signal_power = torch.mean(waveform_orig ** 2)
noise_power = torch.mean((waveform_gen - waveform_orig) ** 2)
snr = 10 * torch.log10(signal_power / noise_power)
print(f"SNR: {snr.item()} dB")

Generating autoregressive samples..


100%|██████████| 16/16 [01:38<00:00,  6.17s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 16/16 [00:18<00:00,  1.19s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [01:42<00:00,  3.89it/s]


In [ ]:
# Additional imports for training
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch.utils.data.dataloader import default_collate  # Import default_collate

# Define TTSDataset class
class TTSDataset(Dataset):
    def __init__(self, text_data, voice_samples, conditioning_latents):
        self.text_data = text_data
        self.voice_samples = voice_samples
        self.conditioning_latents = conditioning_latents

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        try:
            text = self.text_data[idx]
            voice = self.voice_samples[idx]
            latent = self.conditioning_latents[idx]

            return {
                'text': text,
                'voice': voice,
                'latent': latent
            }
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            return None

# Define a collate_fn to filter out None values
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch:  # If batch is empty after filtering
        return None
    return default_collate(batch)

# Prepare dataset
text_data = [
    "The quick brown fox jumps over the lazy dog.",
    "How razorback-jumping frogs can level six piqued gymnasts!",
    "The five boxing wizards jump quickly.",
    "Pack my box with five dozen liquor jugs."
]
voice_samples = [torch.randn(1, 24000), torch.randn(1, 24000), torch.randn(1, 24000), torch.randn(1, 24000)]
conditioning_latents = [torch.randn(1, 24000), torch.randn(1, 24000), torch.randn(1, 24000), torch.randn(1, 24000)]

dataset = TTSDataset(text_data, voice_samples, conditioning_latents)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)


RMSE: 1.469796895980835
SNR: -3.345174551010132 dB


In [ ]:
# Initialize model
class SimpleTTSModel(nn.Module):
    def __init__(self):
        super(SimpleTTSModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(24000, 512),  # Assuming input size 24000 (adjust as per your input)
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 24000)  # Adjusted output size to match your TTS requirements
        )

    def forward(self, input_tensor):
        x = self.encoder(input_tensor)
        x = self.decoder(x)
        return x

model = SimpleTTSModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
        if batch is None:  # Skip empty batches
            continue
        voice = batch['voice']
        latent = batch['latent']

        # Move data to the appropriate device
        voice = voice.to(device)
        latent = latent.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(voice)

        # Calculate loss
        loss = criterion(outputs, latent)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

print("Training complete.")

# Save the model
torch.save(model.state_dict(), 'tts_model.pth')

# Load the model (when needed)
model.load_state_dict(torch.load('tts_model.pth'))
model.eval()

Epoch [1/10], Loss: 0.9937
Epoch [2/10], Loss: 1.5402
Epoch [3/10], Loss: 0.9504
Epoch [4/10], Loss: 0.8128
Epoch [5/10], Loss: 0.6652
Epoch [6/10], Loss: 0.5260
Epoch [7/10], Loss: 0.3998
Epoch [8/10], Loss: 0.3195
Epoch [9/10], Loss: 0.2256
Epoch [10/10], Loss: 0.1714
Training complete.


SimpleTTSModel(
  (encoder): Sequential(
    (0): Linear(in_features=24000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=24000, bias=True)
  )
)

In [ ]:
import torch
import torchaudio
from torchaudio.transforms import Resample
import os
from jiwer import wer

def evaluate_single_example(model, processor, tts, text, voice_sample, custom_voice_folder, preset):
    model.eval()

    # Generate speech with the model
    voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    output_audio_path = f'generated-{CUSTOM_VOICE_NAME}.wav'
    torchaudio.save(output_audio_path, gen.squeeze(0).cpu(), 24000)

    # Load and normalize generated audio using Wav2Vec2 processor
    waveform_gen, sample_rate_gen = torchaudio.load(output_audio_path)
    input_values_gen = processor(waveform_gen.squeeze(0), return_tensors="pt").input_values  # Preprocess with Wav2Vec2 processor
    normalized_waveform_gen = (input_values_gen - input_values_gen.mean()) / input_values_gen.std()

    # Load and normalize original voice sample using Wav2Vec2 processor
    original_audio_path = os.path.join(custom_voice_folder, '0.wav')
    waveform_orig, sample_rate_orig = torchaudio.load(original_audio_path)
    input_values_orig = processor(waveform_orig.squeeze(0), return_tensors="pt").input_values  # Preprocess with Wav2Vec2 processor
    normalized_waveform_orig = (input_values_orig - input_values_orig.mean()) / input_values_orig.std()

    # Debug statements
    print(f"Generated waveform shape: {normalized_waveform_gen.size()}")
    print(f"Original waveform shape: {normalized_waveform_orig.size()}")

    # Perform text-to-text evaluation
    # For simplicity, assume perfect text generation in this example
    predicted_text = text
    target_text = text

    # Calculate WER
    word_error_rate = wer(target_text, predicted_text)

    return word_error_rate, predicted_text, target_text

# Example usage
text = "The quick brown fox jumps over the lazy dog."
voice_sample = torch.randn(1, 24000)  # Example voice sample
word_error_rate, predicted_text, target_text = evaluate_single_example(model, processor, tts, text, voice_sample, custom_voice_folder, preset)

print(f"Word Error Rate: {word_error_rate}")
print(f"Predicted Text: {predicted_text}")
print(f"Target Text: {target_text}")
